# Turn Barrel Explorer

Analyse turn double barrels and delayed c-bets by hand strength, bet sizing, and opponent responses.
Adjust the configuration below to experiment with alternative sizing buckets or grouping schemes.

> Buckets use left-inclusive, right-exclusive ranges (e.g., a 0.25 bet falls into [0.25, 0.40)).


In [1]:
from pathlib import Path
import os

def _locate_project_root() -> Path:
    current = Path().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "AGENTS.md").exists():
            return candidate
    raise FileNotFoundError("Repository root not found from notebook location.")

PROJECT_ROOT = _locate_project_root()
del _locate_project_root

DB_CANDIDATES = [
    Path(r"T:\Dev\ignition\drivehud\drivehud.db"),
    Path("/mnt/t/Dev/ignition/drivehud/drivehud.db"),
    PROJECT_ROOT / "drivehud" / "drivehud.db",
]

for candidate in DB_CANDIDATES:
    if candidate.exists():
        DB_PATH = candidate
        break
else:
    checked = os.linesep.join(str(p) for p in DB_CANDIDATES)
    message = "Database not found. Checked:" + os.linesep + checked
    raise FileNotFoundError(message)

CACHE_PATH = PROJECT_ROOT / "analysis" / "cache" / "turn_events.json"

if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found at {DB_PATH}")


In [2]:
import sys
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

from analysis.turn_utils import (
    TURN_BUCKETS,
    available_primary_categories as available_turn_categories,
    load_turn_events,
    load_turn_first_actions,
    turn_response_events,
)
from analysis.cbet_utils import BASE_PRIMARY_CATEGORIES


In [3]:
# --- Configuration ---
RAW_BUCKET_BOUNDS = [
    (0.00, 0.25),
    (0.25, 0.40),
    (0.40, 0.60),
    (0.60, 0.80),
    (0.80, 1.00),
    (1.00, 1.25),
    (1.25, float('inf')),
]

def _bucket_label(low: float, high: float) -> str:
    if high == float('inf'):
        return f">={low:.2f}"
    return f"[{low:.2f}, {high:.2f})"

TURN_BUCKETS = [(low, high, _bucket_label(low, high)) for (low, high) in RAW_BUCKET_BOUNDS]
del _bucket_label

PRIMARY_GROUPS = {cat: [cat] for cat in BASE_PRIMARY_CATEGORIES}
GROUPED_PRIMARY_ORDER = [
    ('Events', None),
    ('Air', ['Air']),
    ('Weak Pair', ['Underpair', 'Bottom Pair', 'Middle Pair']),
    ('Top Pair', ['Top Pair']),
    ('Overpair', ['Overpair']),
    ('Two Pair', ['Two Pair']),
    ('Trips/Set', ['Trips/Set']),
    ('Monster', ['Straight', 'Flush', 'Full House', 'Quads']),
    ('Draw', ['Flush Draw', 'OESD/DG']),
]

FORCE_RELOAD = True



In [4]:
events = load_turn_events(DB_PATH, cache_path=CACHE_PATH, force=FORCE_RELOAD)
print(f'Loaded {len(events)} turn bet events.')
print('Line types:', ', '.join(sorted({e["line_type"] for e in events})))
available_categories = available_turn_categories(events)
print('Observed primary categories:', ', '.join(available_categories))

events_df = pd.DataFrame(events)
responses_df = pd.DataFrame(turn_response_events(events))
bucket_edges = [bucket[0] for bucket in TURN_BUCKETS] + [TURN_BUCKETS[-1][1]]
bucket_labels = [bucket[2] for bucket in TURN_BUCKETS]
events_df['bucket'] = pd.cut(
    events_df['ratio'],
    bins=bucket_edges,
    labels=bucket_labels,
    right=False,
    include_lowest=True,
)



Loaded 6776 turn bet events.
Line types: Barrel, Delayed, Other
Observed primary categories: Air, Bottom Pair, Flush, Full House, Middle Pair, Overpair, Quads, Straight, Top Pair, Trips/Set, Two Pair, Underpair


In [5]:
DRAW_FIELDS = [('Flush Draw', 'has_flush_draw'), ('OESD/DG', 'has_oesd_dg')]
RESPONSE_DRAW_FIELDS = [('Flush Draw', 'responder_has_flush_draw'), ('OESD/DG', 'responder_has_oesd_dg')]
HAND_ORDER = BASE_PRIMARY_CATEGORIES
LINE_DISPLAY_ORDER = ['Barrel', 'Delayed', 'Other']
SPECIAL_FIELD_CANDIDATES = {
    'All-In': ('is_all_in', 'is_all_in_bet', 'is_all_in_cbet'),
    '1 BB': ('is_one_bb', 'is_one_bb_bet', 'is_one_bb_cbet'),
}

def _gradient_cmap_turn(base_color: str) -> LinearSegmentedColormap:
    name = f'turn_gradient_{base_color.strip("#")}'
    return LinearSegmentedColormap.from_list(name, ['#ffffff', base_color])


def _percent(value, total):
    return (value / total * 100) if total else 0.0

def _select_flag_field(df, candidates):
    for field in candidates:
        if field in df.columns:
            return field
    return None

def _subset_for_flag(df, candidates):
    field = _select_flag_field(df, candidates)
    if field is None:
        return None
    subset = df[df[field]]
    return subset if not subset.empty else None

def summarize_line(df):
    total = len(df)
    summary = {'Events': float(total)}
    for cat in HAND_ORDER:
        summary[cat] = df['primary'].eq(cat).mean() * 100 if total else 0.0
    for label, field in DRAW_FIELDS:
        summary[label] = df[field].mean() * 100 if total else 0.0
    return summary

def summarize_line_grouped(df):
    total = len(df)
    grouped = {'Events': float(total)}
    for label, members in GROUPED_PRIMARY_ORDER:
        if members is None or label == 'Events':
            continue
        if label == 'Draw':
            draws = df['has_flush_draw'].fillna(False) | df['has_oesd_dg'].fillna(False)
            grouped[label] = draws.mean() * 100 if total else 0.0
        else:
            grouped[label] = df['primary'].isin(members).mean() * 100 if total else 0.0
    return grouped

def display_line_comparison(df, caption):
    data = {}
    for line in LINE_DISPLAY_ORDER:
        subset = df[df['line_type'] == line]
        if subset.empty:
            continue
        data[line] = summarize_line(subset)
    if not data:
        print('No data for requested lines.')
        return
    summary_df = pd.DataFrame(data)
    summary_df = summary_df.reindex(['Events'] + HAND_ORDER + [label for label, _ in DRAW_FIELDS])
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='YlOrRd', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(caption)
    )
    display(styled)

def display_line_comparison_grouped(df, caption):
    data = {}
    for line in LINE_DISPLAY_ORDER:
        subset = df[df['line_type'] == line]
        if subset.empty:
            continue
        data[line] = summarize_line_grouped(subset)
    if not data:
        print('No data for requested lines.')
        return
    summary_df = pd.DataFrame(data)
    summary_df = summary_df.reindex(['Events'] + [label for label, members in GROUPED_PRIMARY_ORDER if members is not None])
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='YlOrRd', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(caption)
    )
    display(styled)

def summarize_bucket(df):
    total = len(df)
    result = {'Events': float(total)}
    for cat in HAND_ORDER:
        result[cat] = df['primary'].eq(cat).mean() * 100 if total else 0.0
    for label, field in DRAW_FIELDS:
        result[label] = df[field].mean() * 100 if total else 0.0
    return result

def _style_bucket_table(summary_df, caption, special_cols):
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='YlOrRd', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(caption)
    )
    if special_cols:
        first = special_cols[0]
        idx = summary_df.columns.get_loc(first)
        styled = styled.set_table_styles([
            {'selector': f'th.col_heading.level0.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
            {'selector': f'td.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
        ], overwrite=False)
    return styled



def summarize_bucket_grouped(df):
    total = len(df)
    grouped = {'Events': float(total)}
    for label, members in GROUPED_PRIMARY_ORDER:
        if members is None or label == 'Events':
            continue
        if label == 'Draw':
            draws = df['has_flush_draw'].fillna(False) | df['has_oesd_dg'].fillna(False)
            grouped[label] = draws.mean() * 100 if total else 0.0
        else:
            grouped[label] = df['primary'].isin(members).mean() * 100 if total else 0.0
    return grouped

def _style_bucket_table_grouped(summary_df, caption, special_cols):
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='YlOrRd', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(caption)
    )
    if special_cols:
        first = special_cols[0]
        idx = summary_df.columns.get_loc(first)
        styled = styled.set_table_styles([
            {'selector': f'th.col_heading.level0.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
            {'selector': f'td.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
        ], overwrite=False)
    return styled

def display_bucket_table_grouped(df, caption):
    bucket_labels_local = [bucket[2] for bucket in TURN_BUCKETS]
    rows = []
    index = []
    for bucket in bucket_labels_local:
        subset = df[df['bucket'] == bucket]
        if subset.empty:
            continue
        rows.append(summarize_bucket_grouped(subset))
        index.append(bucket)
    if not rows:
        print('No data for selected line.')
        return
    summary_df = pd.DataFrame(rows, index=index).T
    summary_df = summary_df.reindex(['Events'] + [label for label, members in GROUPED_PRIMARY_ORDER if members is not None])
    special_cols = []
    for label, candidates in SPECIAL_FIELD_CANDIDATES.items():
        subset = _subset_for_flag(df, candidates)
        if subset is None:
            continue
        summary_df[label] = pd.Series(summarize_bucket_grouped(subset))
        special_cols.append(label)
    ordered_columns = [col for col in bucket_labels_local if col in summary_df.columns]
    ordered_columns += [col for col in special_cols if col in summary_df.columns]
    summary_df = summary_df[ordered_columns]
    styled = _style_bucket_table_grouped(summary_df, caption, special_cols)
    display(styled)
def display_bucket_table(df, caption):
    bucket_labels_local = [bucket[2] for bucket in TURN_BUCKETS]
    rows = []
    index = []
    for bucket in bucket_labels_local:
        subset = df[df['bucket'] == bucket]
        if subset.empty:
            continue
        rows.append(summarize_bucket(subset))
        index.append(bucket)
    if not rows:
        print('No data for selected line.')
        return
    summary_df = pd.DataFrame(rows, index=index).T
    summary_df = summary_df.reindex(['Events'] + HAND_ORDER + [label for label, _ in DRAW_FIELDS])
    special_cols = []
    for label, candidates in SPECIAL_FIELD_CANDIDATES.items():
        subset = _subset_for_flag(df, candidates)
        if subset is None:
            continue
        summary_df[label] = pd.Series(summarize_bucket(subset))
        special_cols.append(label)
    ordered_columns = [col for col in bucket_labels_local if col in summary_df.columns]
    ordered_columns += [col for col in special_cols if col in summary_df.columns]
    summary_df = summary_df[ordered_columns]
    styled = _style_bucket_table(summary_df, caption, special_cols)
    display(styled)



def summarize_responses_grouped(df, response_type):
    df = df[df['response'] == response_type].dropna(subset=['responder_primary'])
    if df.empty:
        return None
    data = {}
    for line in ['Barrel', 'Delayed']:
        subset = df[df['line_type'] == line]
        if subset.empty:
            continue
        total = len(subset)
        entry = {'Events': float(total)}
        for label, members in GROUPED_PRIMARY_ORDER:
            if members is None or label == 'Events':
                continue
            if label == 'Draw':
                draws = subset['responder_has_flush_draw'].fillna(False) | subset['responder_has_oesd_dg'].fillna(False)
                entry[label] = draws.mean() * 100 if total else 0.0
            else:
                entry[label] = subset['responder_primary'].isin(members).mean() * 100 if total else 0.0
        data[line] = entry
    if not data:
        return None
    summary_df = pd.DataFrame(data)
    summary_df = summary_df.reindex(['Events'] + [label for label, members in GROUPED_PRIMARY_ORDER if members is not None])
    return summary_df

def _style_response_table_grouped(summary_df, caption):
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='PuBu', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(caption)
    )
    return styled
def summarize_responses(df, response_type):
    df = df[df['response'] == response_type].dropna(subset=['responder_primary'])
    if df.empty:
        return None
    data = {}
    for line in ['Barrel', 'Delayed']:
        subset = df[df['line_type'] == line]
        if subset.empty:
            continue
        total = len(subset)
        entry = {'Events': float(total)}
        for cat in HAND_ORDER:
            entry[cat] = subset['responder_primary'].eq(cat).mean() * 100 if total else 0.0
        for label, field in RESPONSE_DRAW_FIELDS:
            entry[label] = subset[field].mean() * 100 if total else 0.0
        data[line] = entry
    if not data:
        return None
    summary_df = pd.DataFrame(data)
    summary_df = summary_df.reindex(['Events'] + HAND_ORDER + [label for label, _ in RESPONSE_DRAW_FIELDS])
    return summary_df

def display_response_table(response_type):
    summary_df = summarize_responses(responses_df, response_type)
    if summary_df is None:
        print(f'No responder data for {response_type} events.')
        return
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='PuBu', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption(f'{response_type} vs Turn Bets (Responder Holdings)')
    )
    display(styled)



def summarize_responses_df_turn(responses_df, buckets=TURN_BUCKETS):
    summary = []
    for low, high, label in buckets:
        subset = responses_df[(responses_df['ratio'] >= low) & (responses_df['ratio'] < high)]
        if subset.empty:
            summary.append({
                'Bucket': label,
                'Events': 0,
                'Fold': 0.0,
                'Call': 0.0,
                'Raise': 0.0,
                'Continue': 0.0,
            })
            continue
        events_count = len(subset)
        response_counts = subset['response'].value_counts()
        continue_count = int(response_counts.get('Call', 0) + response_counts.get('Raise', 0))
        summary.append({
            'Bucket': label,
            'Events': events_count,
            'Fold': response_counts.get('Fold', 0) / events_count * 100,
            'Call': response_counts.get('Call', 0) / events_count * 100,
            'Raise': response_counts.get('Raise', 0) / events_count * 100,
            'Continue': continue_count / events_count * 100,
        })
    return summary

def _prepare_response_table_turn(records):
    if not records:
        return pd.DataFrame()
    df = pd.DataFrame(records)
    if 'Bucket' not in df.columns:
        return pd.DataFrame()
    df = df.set_index('Bucket').T.apply(pd.to_numeric, errors='coerce')
    row_order = ['Events', 'Fold', 'Call', 'Raise', 'Continue']
    ordered = [row for row in row_order if row in df.index]
    extras = [row for row in df.index if row not in row_order]
    if ordered or extras:
        df = df.loc[ordered + extras]
    return df

def _response_summary_turn(subset_df):
    if subset_df is None or subset_df.empty:
        return {
            'Events': 0,
            'Fold': 0.0,
            'Call': 0.0,
            'Raise': 0.0,
            'Continue': 0.0,
        }
    events_count = len(subset_df)
    response_counts = subset_df['response'].value_counts()
    continue_count = int(response_counts.get('Call', 0) + response_counts.get('Raise', 0))
    return {
        'Events': events_count,
        'Fold': response_counts.get('Fold', 0) / events_count * 100,
        'Call': response_counts.get('Call', 0) / events_count * 100,
        'Raise': response_counts.get('Raise', 0) / events_count * 100,
        'Continue': continue_count / events_count * 100,
    }

def _compute_special_columns_turn(source_df):
    specials = {}
    if source_df is None:
        return specials
    if 'is_all_in_bet' in source_df.columns:
        specials['All-In'] = _response_summary_turn(source_df[source_df['is_all_in_bet']])
    if 'is_one_bb_bet' in source_df.columns:
        specials['1 BB'] = _response_summary_turn(source_df[source_df['is_one_bb_bet']])
    return specials

def _style_response_table_turn(df, title, base_color='#6baed6'):
    if df.empty:
        return None
    percent_rows = [row for row in ['Fold', 'Call', 'Raise', 'Continue'] if row in df.index]
    styled = df.style
    if 'Events' in df.index:
        styled = styled.format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
    if percent_rows:
        styled = styled.format('{:.1f}%', subset=pd.IndexSlice[percent_rows, :])
        styled = styled.background_gradient(
            cmap=_gradient_cmap_turn(base_color),
            axis=None,
            subset=pd.IndexSlice[percent_rows, :]
        )
    special_cols = [col for col in ['All-In', '1 BB'] if col in df.columns]
    if special_cols:
        first = special_cols[0]
        idx = df.columns.get_loc(first)
        styled = styled.set_table_styles([
            {'selector': f'th.col_heading.level0.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
            {'selector': f'td.col{idx}', 'props': [('border-left', '2px solid #64748b')]},
        ], overwrite=False)
    return styled.set_caption(title)

def display_response_tables_turn(summary_records, title, base_color='#6baed6', source_df=None):
    df = _prepare_response_table_turn(summary_records)
    if df.empty:
        display(pd.DataFrame({'Bucket': bucket_labels, 'Message': 'No events'}))
        return
    specials = _compute_special_columns_turn(source_df)
    if specials:
        for col_name, values in specials.items():
            for row in df.index:
                df.loc[row, col_name] = values.get(row, 0.0)
    styled = _style_response_table_turn(df, title, base_color=base_color)
    if styled is not None:
        display(styled)

def display_line_response_tables(line_type, title_prefix, base_color='#6baed6'):
    subset = responses_df[responses_df['line_type'] == line_type]
    if subset.empty:
        print(f'No response events recorded for {line_type}.')
        return
    overall = summarize_responses_df_turn(subset, TURN_BUCKETS)
    display_response_tables_turn(overall, f"{title_prefix} (Overall)", base_color=base_color, source_df=subset)

    ip_subset = subset[subset['bettor_in_position']]
    ip_summary = summarize_responses_df_turn(ip_subset, TURN_BUCKETS)
    display_response_tables_turn(ip_summary, f"{title_prefix} (In Position)", base_color=base_color, source_df=ip_subset)

    oop_subset = subset[~subset['bettor_in_position']]
    oop_summary = summarize_responses_df_turn(oop_subset, TURN_BUCKETS)
    display_response_tables_turn(oop_summary, f"{title_prefix} (Out of Position)", base_color=base_color, source_df=oop_subset)


In [6]:
def _filter_standard_turn_events(df):
    if df is None or df.empty:
        return df.iloc[0:0].copy()
    mask = pd.Series(True, index=df.index)
    if 'is_all_in' in df.columns:
        mask &= ~df['is_all_in']
    if 'is_one_bb' in df.columns:
        mask &= ~df['is_one_bb']
    return df.loc[mask].copy()


def _filter_standard_turn_responses(df):
    if df is None or df.empty:
        return df.iloc[0:0].copy()
    mask = pd.Series(True, index=df.index)
    if 'is_all_in_bet' in df.columns:
        mask &= ~df['is_all_in_bet']
    if 'is_one_bb_bet' in df.columns:
        mask &= ~df['is_one_bb_bet']
    return df.loc[mask].copy()


def _safe_mean(series):
    if series is None or len(series) == 0:
        return 0.0
    cleaned = series.dropna()
    if cleaned.empty:
        return 0.0
    value = cleaned.mean()
    return float(value) if pd.notna(value) else 0.0


def _prepare_turn_event_metrics(events_subset, responses_subset, *, exclude_special=True):
    if events_subset is None or events_subset.empty:
        return pd.DataFrame()
    events_work = events_subset.copy()
    responses_work = responses_subset.copy()
    if exclude_special:
        events_work = _filter_standard_turn_events(events_work)
        responses_work = _filter_standard_turn_responses(responses_work)
    if events_work.empty:
        return pd.DataFrame()
    events_work = events_work.rename(columns={'player': 'bettor'})
    base_cols = [
        'hand_number',
        'bettor',
        'line_type',
        'ratio',
        'bucket',
        'bet_amount',
        'bet_amount_bb',
        'big_blind',
        'pot_before',
        'bettor_in_position',
    ]
    missing = [col for col in base_cols if col not in events_work.columns]
    if missing:
        raise KeyError(f"Missing columns in events dataframe: {', '.join(missing)}")
    events_work = events_work[base_cols].copy()
    if responses_work.empty:
        agg = pd.DataFrame(
            columns=[
                'hand_number',
                'bettor',
                'continue_amount',
                'call_amount',
                'raise_amount',
                'call_flag',
                'raise_flag',
            ]
        )
    else:
        prepped = responses_work.copy()
        prepped['continue_amount'] = np.where(
            prepped['response'].isin(['Call', 'Raise']),
            prepped['response_amount'],
            0.0,
        )
        prepped['call_amount'] = np.where(
            prepped['response'] == 'Call',
            prepped['response_amount'],
            0.0,
        )
        prepped['raise_amount'] = np.where(
            prepped['response'] == 'Raise',
            prepped['response_amount'],
            0.0,
        )
        prepped['call_flag'] = (prepped['response'] == 'Call').astype(int)
        prepped['raise_flag'] = (prepped['response'] == 'Raise').astype(int)
        agg = (
            prepped.groupby(['hand_number', 'bettor'], as_index=False)
            .agg({
                'continue_amount': 'sum',
                'call_amount': 'sum',
                'raise_amount': 'sum',
                'call_flag': 'max',
                'raise_flag': 'max',
            })
        )
    metrics = events_work.merge(agg, on=['hand_number', 'bettor'], how='left')
    for column in ['continue_amount', 'call_amount', 'raise_amount']:
        metrics[column] = metrics[column].fillna(0.0)
    for column in ['call_flag', 'raise_flag']:
        metrics[column] = metrics[column].fillna(0).astype(int)
    metrics['continue_flag'] = metrics['continue_amount'] > 0
    metrics['call_only_flag'] = (metrics['call_flag'] > 0) & (metrics['raise_flag'] == 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        metrics['bet_ratio_pct'] = metrics['ratio'].astype(float) * 100.0
        metrics['raise_ratio_pct'] = np.where(
            (metrics['raise_flag'] > 0) & (metrics['pot_before'] > 0),
            (metrics['raise_amount'] / metrics['pot_before']) * 100.0,
            np.nan,
        )
    metrics['breakeven_fold_pct'] = np.where(
        (metrics['bet_amount'] + metrics['pot_before']) > 0,
        metrics['bet_amount'] / (metrics['bet_amount'] + metrics['pot_before']) * 100,
        np.nan,
    )
    return metrics


def _compute_turn_event_bucket_metrics(metrics_df, buckets=TURN_BUCKETS):
    results = []
    if metrics_df is None or metrics_df.empty:
        for _, _, label in buckets:
            results.append({'Bucket': label, 'Events': 0.0})
        return results
    for low, high, label in buckets:
        bucket_mask = (metrics_df['ratio'] >= low) & (metrics_df['ratio'] < high)
        bucket_events = metrics_df.loc[bucket_mask]
        total = len(bucket_events)
        if total == 0:
            results.append({'Bucket': label, 'Events': 0.0})
            continue
        continue_events = int(bucket_events['continue_flag'].sum())
        raise_events = int(bucket_events['raise_flag'].sum())
        call_only_events = int(bucket_events['call_only_flag'].sum())
        fold_pct = (total - continue_events) / total * 100
        continue_pct = continue_events / total * 100
        raise_pct = raise_events / total * 100
        call_only_pct = call_only_events / total * 100
        avg_ratio_pct = _safe_mean(bucket_events['bet_ratio_pct'])
        raise_mask = bucket_events['raise_flag'] > 0
        avg_raise_ratio_pct = _safe_mean(bucket_events.loc[raise_mask, 'raise_ratio_pct'])
        avg_breakeven = _safe_mean(bucket_events['breakeven_fold_pct'])
        expected_call_contrib_pct = (call_only_pct * avg_ratio_pct) / 100.0
        expected_raise_contrib_pct = (raise_pct * avg_raise_ratio_pct) / 100.0
        total_contrib_pct = expected_call_contrib_pct + expected_raise_contrib_pct
        row = {
            'Bucket': label,
            'Events': float(total),
            'Fold%': fold_pct,
            'Continue%': continue_pct,
            'Call-Only%': call_only_pct,
            'Raise%': raise_pct,
            'Avg Size (Pot%)': avg_ratio_pct,
            'Avg Raise Size (Pot%)': avg_raise_ratio_pct,
            'Expected Call Contribution (Pot%)': expected_call_contrib_pct,
            'Expected Raise Contribution (Pot%)': expected_raise_contrib_pct,
            'Raise Contribution Total (Pot%)': total_contrib_pct,
            'Breakeven Fold%': avg_breakeven,
        }
        row['Fold Surplus'] = row['Fold%'] - row['Breakeven Fold%']
        results.append(row)
    return results


VALUE_COLUMNS_TURN = [
    'Bucket',
    'Events',
    'Continue%',
    'Call-Only%',
    'Raise%',
    'Avg Size (Pot%)',
    'Avg Raise Size (Pot%)',
    'Expected Call Contribution (Pot%)',
    'Expected Raise Contribution (Pot%)',
    'Raise Contribution Total (Pot%)',
]


PRESSURE_COLUMNS_TURN = [
    'Bucket',
    'Events',
    'Fold%',
    'Avg Size (Pot%)',
    'Breakeven Fold%',
    'Fold Surplus',
]


_FOLD_SURPLUS_CMAP = LinearSegmentedColormap.from_list(
    'fold_surplus_cmp', ['#b91c1c', '#ffffff', '#15803d']
)


def _style_turn_value_table(table_df, title):
    if table_df.empty:
        return None
    df = table_df.set_index('Bucket').T
    percent_rows = [row for row in df.index if row != 'Events']
    styled = df.style
    if 'Events' in df.index:
        styled = styled.format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
    if percent_rows:
        styled = styled.format('{:.1f}%', subset=pd.IndexSlice[percent_rows, :])
        contribution_rows = [row for row in percent_rows if 'Contribution' in row]
        size_rows = [row for row in percent_rows if 'Size' in row]
        pct_rows_other = [row for row in percent_rows if row not in contribution_rows + size_rows]
        if pct_rows_other:
            styled = styled.background_gradient(
                cmap='YlOrRd',
                axis=None,
                subset=pd.IndexSlice[pct_rows_other, :],
            )
        if size_rows:
            styled = styled.background_gradient(
                cmap='BuGn',
                axis=None,
                subset=pd.IndexSlice[size_rows, :],
            )
        if contribution_rows:
            styled = styled.background_gradient(
                cmap='PuBu',
                axis=None,
                subset=pd.IndexSlice[contribution_rows, :],
            )
    return styled.set_caption(title)


def _style_turn_pressure_table(table_df, title):
    if table_df.empty:
        return None
    df = table_df.set_index('Bucket').T
    desired_order = ['Events', 'Fold%', 'Avg Size (Pot%)', 'Breakeven Fold%', 'Fold Surplus']
    existing_order = [row for row in desired_order if row in df.index]
    df = df.loc[existing_order]
    styled = df.style
    if 'Events' in df.index:
        styled = styled.format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
    percent_rows = [row for row in df.index if row != 'Events']
    if percent_rows:
        styled = styled.format('{:.1f}%', subset=pd.IndexSlice[percent_rows, :])
    if 'Fold%' in df.index:
        styled = styled.background_gradient(
            cmap='YlOrRd',
            axis=None,
            subset=pd.IndexSlice[['Fold%'], :],
        )
    if 'Avg Size (Pot%)' in df.index:
        styled = styled.background_gradient(
            cmap='BuGn',
            axis=None,
            subset=pd.IndexSlice[['Avg Size (Pot%)'], :],
        )
    if 'Fold Surplus' in df.index:
        fold_values = df.loc['Fold Surplus'].dropna().astype(float)
        if not fold_values.empty:
            max_abs = max(abs(fold_values.min()), abs(fold_values.max()))
            if max_abs == 0:
                max_abs = 1.0
            styled = styled.background_gradient(
                cmap=_FOLD_SURPLUS_CMAP,
                axis=None,
                subset=pd.IndexSlice[['Fold Surplus'], :],
                vmin=-max_abs,
                vmax=max_abs,
            )
    return styled.set_caption(title)


def _display_turn_event_tables(metrics_df, title_prefix):
    if metrics_df is None or metrics_df.empty:
        print(f'No events available for {title_prefix}.')
        return
    records = _compute_turn_event_bucket_metrics(metrics_df)
    if not records:
        print(f'No bucketed metrics available for {title_prefix}.')
        return
    metrics_table = pd.DataFrame(records)
    value_columns = [col for col in VALUE_COLUMNS_TURN if col in metrics_table.columns]
    pressure_columns = [col for col in PRESSURE_COLUMNS_TURN if col in metrics_table.columns]
    value_table = metrics_table[value_columns]
    pressure_table = metrics_table[pressure_columns]
    value_styled = _style_turn_value_table(value_table, f"{title_prefix} - Value Profile")
    if value_styled is not None:
        display(value_styled)
    pressure_styled = _style_turn_pressure_table(pressure_table, f"{title_prefix} - Fold Pressure")
    if pressure_styled is not None:
        display(pressure_styled)


def display_turn_event_ev_tables(line_type, title_prefix, *, exclude_special=True):
    line_events = events_df[events_df['line_type'] == line_type]
    if line_events.empty:
        print(f'No {line_type.lower()} events available for analysis.')
        return
    line_responses = responses_df[responses_df['line_type'] == line_type]
    base_metrics = _prepare_turn_event_metrics(line_events, line_responses, exclude_special=exclude_special)
    if base_metrics.empty:
        print(f'No {line_type.lower()} events remain after filtering special cases.')
        return
    subsets = [
        (base_metrics, f"{title_prefix} (Overall)"),
        (base_metrics[base_metrics['bettor_in_position']], f"{title_prefix} (In Position)"),
        (base_metrics[~base_metrics['bettor_in_position']], f"{title_prefix} (Out of Position)"),
    ]
    for subset_df, label in subsets:
        if subset_df is None or subset_df.empty:
            continue
        _display_turn_event_tables(subset_df, label)


def display_all_turn_event_tables():
    if responses_df.empty or events_df.empty:
        print('No turn events or responses available for event-level contribution tables.')
        return
    display_turn_event_ev_tables('Barrel', 'Turn Barrel Event Outcomes')
    display_turn_event_ev_tables('Delayed', 'Turn Delayed C-Bet Event Outcomes')


In [7]:
display_line_comparison(events_df, 'Turn Hand Types by Line')
display_line_comparison_grouped(events_df, 'Turn Hand Types by Line (Grouped)')


,Barrel,Delayed,Other
Events,3140,2843,793
Air,28.0%,39.2%,30.8%
Underpair,4.2%,4.3%,4.4%
Bottom Pair,3.3%,4.6%,4.0%
Middle Pair,7.6%,7.1%,8.1%
Top Pair,17.7%,13.1%,13.9%
Overpair,4.3%,0.8%,1.6%
Two Pair,20.0%,16.4%,19.4%
Trips/Set,6.9%,7.1%,6.9%
Straight,3.7%,3.4%,6.1%


,Barrel,Delayed,Other
Events,3140,2843,793
Air,28.0%,39.2%,30.8%
Weak Pair,15.0%,16.0%,16.5%
Top Pair,17.7%,13.1%,13.9%
Overpair,4.3%,0.8%,1.6%
Two Pair,20.0%,16.4%,19.4%
Trips/Set,6.9%,7.1%,6.9%
Monster,8.1%,7.5%,10.8%
Draw,21.4%,21.5%,22.3%


In [8]:
display_bucket_table(events_df[events_df['line_type'] == 'Barrel'], 'Turn Bet Sizing vs Hand Type (Barrel)')
display_bucket_table_grouped(events_df[events_df['line_type'] == 'Barrel'], 'Turn Bet Sizing vs Hand Type (Barrel) — Grouped')
display_bucket_table(events_df[events_df['line_type'] == 'Delayed'], 'Turn Bet Sizing vs Hand Type (Delayed)')
display_bucket_table_grouped(events_df[events_df['line_type'] == 'Delayed'], 'Turn Bet Sizing vs Hand Type (Delayed) — Grouped')


,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,348,397,823,927,399,68,178,208,200
Air,29.6%,23.4%,29.8%,30.1%,20.8%,30.9%,30.3%,17.3%,33.0%
Underpair,5.2%,5.8%,4.9%,3.5%,2.5%,1.5%,3.9%,4.8%,2.5%
Bottom Pair,8.0%,5.0%,3.3%,1.7%,2.5%,1.5%,0.6%,1.4%,11.5%
Middle Pair,8.0%,11.1%,9.8%,5.3%,7.8%,2.9%,1.7%,7.2%,8.0%
Top Pair,13.8%,16.4%,17.0%,20.0%,17.8%,14.7%,21.3%,21.6%,13.0%
Overpair,1.4%,3.0%,3.8%,5.8%,4.8%,10.3%,3.4%,10.1%,0.0%
Two Pair,23.3%,20.9%,19.3%,17.7%,22.3%,20.6%,20.8%,24.5%,23.5%
Trips/Set,6.3%,7.1%,5.2%,8.0%,7.8%,10.3%,6.7%,5.3%,5.0%
Straight,2.3%,3.0%,2.9%,3.6%,6.8%,4.4%,5.6%,3.4%,1.0%


,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,348,397,823,927,399,68,178,208,200
Air,29.6%,23.4%,29.8%,30.1%,20.8%,30.9%,30.3%,17.3%,33.0%
Weak Pair,21.3%,21.9%,18.0%,10.5%,12.8%,5.9%,6.2%,13.5%,22.0%
Top Pair,13.8%,16.4%,17.0%,20.0%,17.8%,14.7%,21.3%,21.6%,13.0%
Overpair,1.4%,3.0%,3.8%,5.8%,4.8%,10.3%,3.4%,10.1%,0.0%
Two Pair,23.3%,20.9%,19.3%,17.7%,22.3%,20.6%,20.8%,24.5%,23.5%
Trips/Set,6.3%,7.1%,5.2%,8.0%,7.8%,10.3%,6.7%,5.3%,5.0%
Monster,4.3%,7.3%,6.9%,8.0%,13.8%,7.4%,11.2%,7.7%,3.5%
Draw,21.6%,23.4%,21.9%,21.7%,18.0%,23.5%,20.2%,17.8%,22.5%


,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,150,474,1034,633,406,41,105,28,366
Air,47.3%,36.7%,41.9%,38.7%,34.2%,41.5%,34.3%,25.0%,49.7%
Underpair,5.3%,6.8%,4.9%,3.2%,2.0%,0.0%,1.9%,3.6%,2.7%
Bottom Pair,2.7%,4.4%,5.9%,3.5%,4.7%,4.9%,1.9%,0.0%,6.0%
Middle Pair,8.0%,6.8%,7.9%,6.6%,7.1%,7.3%,2.9%,7.1%,7.1%
Top Pair,9.3%,12.9%,11.2%,16.1%,15.0%,12.2%,12.4%,25.0%,7.4%
Overpair,2.7%,1.1%,0.5%,0.8%,1.0%,0.0%,0.0%,0.0%,0.3%
Two Pair,16.0%,17.5%,15.6%,17.9%,14.3%,12.2%,20.0%,25.0%,15.0%
Trips/Set,2.7%,6.1%,6.2%,6.6%,10.8%,12.2%,12.4%,7.1%,5.7%
Straight,2.0%,2.3%,3.0%,3.0%,5.2%,7.3%,9.5%,3.6%,3.6%


,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,150,474,1034,633,406,41,105,28,366
Air,47.3%,36.7%,41.9%,38.7%,34.2%,41.5%,34.3%,25.0%,49.7%
Weak Pair,16.0%,17.9%,18.8%,13.3%,13.8%,12.2%,6.7%,10.7%,15.8%
Top Pair,9.3%,12.9%,11.2%,16.1%,15.0%,12.2%,12.4%,25.0%,7.4%
Overpair,2.7%,1.1%,0.5%,0.8%,1.0%,0.0%,0.0%,0.0%,0.3%
Two Pair,16.0%,17.5%,15.6%,17.9%,14.3%,12.2%,20.0%,25.0%,15.0%
Trips/Set,2.7%,6.1%,6.2%,6.6%,10.8%,12.2%,12.4%,7.1%,5.7%
Monster,6.0%,7.8%,5.9%,6.6%,10.8%,9.8%,14.3%,7.1%,6.0%
Draw,20.7%,19.2%,21.8%,22.7%,22.7%,24.4%,18.1%,21.4%,23.8%


In [9]:
summary_df = summarize_responses(responses_df, 'Call')
if summary_df is not None:
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='PuBu', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption('Call vs Turn Bets (Responder Holdings)')
    )
    display(styled)
grouped_call = summarize_responses_grouped(responses_df, 'Call')
if grouped_call is not None:
    display(_style_response_table_grouped(grouped_call, 'Call vs Turn Bets (Responder Holdings) — Grouped'))
summary_df = summarize_responses(responses_df, 'Raise')
if summary_df is not None:
    styled = (
        summary_df.style
        .format('{:.0f}', subset=pd.IndexSlice[['Events'], :])
        .format('{:.1f}%', subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .background_gradient(cmap='PuBu', axis=None, subset=pd.IndexSlice[summary_df.index.difference(['Events']), :])
        .set_caption('Raise vs Turn Bets (Responder Holdings)')
    )
    display(styled)
grouped_raise = summarize_responses_grouped(responses_df, 'Raise')
if grouped_raise is not None:
    display(_style_response_table_grouped(grouped_raise, 'Raise vs Turn Bets (Responder Holdings) — Grouped'))


,Barrel,Delayed
Events,1600,1434
Air,34.0%,48.7%
Underpair,5.1%,5.9%
Bottom Pair,5.6%,5.9%
Middle Pair,9.9%,9.0%
Top Pair,17.2%,8.9%
Overpair,1.3%,0.7%
Two Pair,16.3%,13.0%
Trips/Set,4.6%,4.1%
Straight,1.9%,1.4%


,Barrel,Delayed
Events,1600,1434
Air,34.0%,48.7%
Weak Pair,20.6%,20.9%
Top Pair,17.2%,8.9%
Overpair,1.3%,0.7%
Two Pair,16.3%,13.0%
Trips/Set,4.6%,4.1%
Monster,5.9%,3.8%
Draw,33.5%,30.7%


,Barrel,Delayed
Events,515,236
Air,21.9%,27.5%
Underpair,2.5%,1.3%
Bottom Pair,3.7%,1.3%
Middle Pair,4.1%,6.8%
Top Pair,11.7%,15.7%
Overpair,3.7%,1.3%
Two Pair,22.3%,14.0%
Trips/Set,12.0%,14.0%
Straight,9.9%,11.0%


,Barrel,Delayed
Events,515,236
Air,21.9%,27.5%
Weak Pair,10.3%,9.3%
Top Pair,11.7%,15.7%
Overpair,3.7%,1.3%
Two Pair,22.3%,14.0%
Trips/Set,12.0%,14.0%
Monster,18.1%,18.2%
Draw,22.9%,19.1%


In [10]:
if not responses_df.empty:
    display_line_response_tables('Barrel', 'Responses vs Turn Barrel')
    display_line_response_tables('Delayed', 'Responses vs Turn Delayed C-Bet')
else:
    print('No turn responses available for line breakdown.')


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,429,434,908,984,432,69,192,245,237
Fold,13.8%,30.6%,33.1%,45.7%,48.8%,60.9%,71.4%,49.8%,11.4%
Call,61.8%,52.8%,54.7%,41.0%,36.6%,27.5%,15.1%,35.1%,61.6%
Raise,24.5%,16.6%,12.1%,13.3%,14.6%,11.6%,13.5%,15.1%,27.0%
Continue,86.2%,69.4%,66.9%,54.3%,51.2%,39.1%,28.6%,50.2%,88.6%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,170,217,462,584,234,34,116,130,86
Fold,19.4%,37.3%,39.4%,49.3%,57.7%,61.8%,74.1%,56.2%,15.1%
Call,62.9%,47.0%,51.3%,37.7%,32.5%,32.4%,15.5%,32.3%,68.6%
Raise,17.6%,15.7%,9.3%,13.0%,9.8%,5.9%,10.3%,11.5%,16.3%
Continue,80.6%,62.7%,60.6%,50.7%,42.3%,38.2%,25.9%,43.8%,84.9%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,259,217,446,400,198,35,76,115,151
Fold,10.0%,24.0%,26.7%,40.5%,38.4%,60.0%,67.1%,42.6%,9.3%
Call,61.0%,58.5%,58.3%,45.8%,41.4%,22.9%,14.5%,38.3%,57.6%
Raise,29.0%,17.5%,15.0%,13.8%,20.2%,17.1%,18.4%,19.1%,33.1%
Continue,90.0%,76.0%,73.3%,59.5%,61.6%,40.0%,32.9%,57.4%,90.7%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,222,678,1336,866,540,59,137,40,481
Fold,34.7%,45.0%,55.5%,61.2%,67.8%,76.3%,75.2%,72.5%,45.9%
Call,51.4%,46.2%,38.8%,34.8%,28.3%,22.0%,15.3%,15.0%,44.7%
Raise,14.0%,8.8%,5.6%,4.0%,3.9%,1.7%,9.5%,12.5%,9.4%
Continue,65.3%,55.0%,44.5%,38.8%,32.2%,23.7%,24.8%,27.5%,54.1%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,81,332,610,401,257,41,69,18,218
Fold,49.4%,47.9%,61.3%,70.1%,74.7%,80.5%,82.6%,72.2%,56.0%
Call,40.7%,45.8%,35.2%,28.2%,23.7%,19.5%,14.5%,22.2%,40.4%
Raise,9.9%,6.3%,3.4%,1.7%,1.6%,0.0%,2.9%,5.6%,3.7%
Continue,50.6%,52.1%,38.7%,29.9%,25.3%,19.5%,17.4%,27.8%,44.0%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25,All-In,1 BB
Events,141,346,726,465,283,18,68,22,263
Fold,26.2%,42.2%,50.7%,53.5%,61.5%,66.7%,67.6%,72.7%,37.6%
Call,57.4%,46.5%,41.9%,40.4%,32.5%,27.8%,16.2%,9.1%,48.3%
Raise,16.3%,11.3%,7.4%,6.0%,6.0%,5.6%,16.2%,18.2%,14.1%
Continue,73.8%,57.8%,49.3%,46.5%,38.5%,33.3%,32.4%,27.3%,62.4%


### Turn Event-Level Outcomes

These tables look at entire betting rounds rather than individual responses.
The Value Profile highlights how much money flows into the pot when villains
continue, while the Fold Pressure view compares actual folds to the break-even
threshold for each bet size. One big blind probes and all-in bets are excluded
to keep the focus on the main sizing buckets.


In [11]:
if events_df.empty or responses_df.empty:
    print('No turn events or responses available for event-level analysis.')
else:
    display_all_turn_event_tables()


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,153,366,800,902,381,46,84
Continue%,88.9%,71.0%,70.5%,56.3%,53.3%,37.0%,22.6%
Call-Only%,58.8%,54.1%,57.4%,42.7%,37.5%,30.4%,11.9%
Raise%,30.1%,16.9%,13.1%,13.6%,15.7%,6.5%,10.7%
Avg Size (Pot%),18.2%,31.9%,48.1%,69.5%,93.3%,113.3%,164.5%
Avg Raise Size (Pot%),69.7%,128.2%,172.6%,227.2%,236.3%,225.0%,374.6%
Expected Call Contribution (Pot%),10.7%,17.3%,27.6%,29.6%,35.0%,34.5%,19.6%
Expected Raise Contribution (Pot%),20.9%,21.7%,22.6%,31.0%,37.2%,14.7%,40.1%
Raise Contribution Total (Pot%),31.6%,39.0%,50.3%,60.6%,72.2%,49.2%,59.7%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,153,366,800,902,381,46,84
Fold%,11.1%,29.0%,29.5%,43.7%,46.7%,63.0%,77.4%
Avg Size (Pot%),18.2%,31.9%,48.1%,69.5%,93.3%,113.3%,164.5%
Breakeven Fold%,15.2%,24.1%,32.5%,41.0%,48.2%,53.1%,61.1%
Fold Surplus,-4.1%,4.8%,-3.0%,2.7%,-1.5%,10.0%,16.3%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,62,180,395,524,205,29,56
Continue%,80.6%,65.6%,64.6%,53.6%,43.9%,37.9%,21.4%
Call-Only%,53.2%,49.4%,54.4%,40.1%,33.2%,34.5%,10.7%
Raise%,27.4%,16.1%,10.1%,13.5%,10.7%,3.4%,10.7%
Avg Size (Pot%),18.6%,32.3%,48.3%,69.7%,93.3%,113.6%,162.0%
Avg Raise Size (Pot%),72.3%,148.2%,163.5%,246.0%,231.8%,242.6%,493.2%
Expected Call Contribution (Pot%),9.9%,16.0%,26.3%,27.9%,31.0%,39.2%,17.4%
Expected Raise Contribution (Pot%),19.8%,23.9%,16.6%,33.3%,24.9%,8.4%,52.8%
Raise Contribution Total (Pot%),29.7%,39.9%,42.8%,61.3%,55.8%,47.5%,70.2%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,62,180,395,524,205,29,56
Fold%,19.4%,34.4%,35.4%,46.4%,56.1%,62.1%,78.6%
Avg Size (Pot%),18.6%,32.3%,48.3%,69.7%,93.3%,113.6%,162.0%
Breakeven Fold%,15.5%,24.4%,32.5%,41.0%,48.2%,53.1%,61.0%
Fold Surplus,3.9%,10.1%,2.9%,5.3%,7.9%,8.9%,17.5%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,91,186,405,378,176,17,28
Continue%,94.5%,76.3%,76.3%,60.1%,64.2%,35.3%,25.0%
Call-Only%,62.6%,58.6%,60.2%,46.3%,42.6%,23.5%,14.3%
Raise%,31.9%,17.7%,16.0%,13.8%,21.6%,11.8%,10.7%
Avg Size (Pot%),17.9%,31.5%,48.0%,69.1%,93.3%,112.8%,169.7%
Avg Raise Size (Pot%),68.1%,110.6%,178.1%,201.5%,238.8%,216.2%,137.3%
Expected Call Contribution (Pot%),11.2%,18.4%,28.9%,32.0%,39.7%,26.5%,24.2%
Expected Raise Contribution (Pot%),21.7%,19.6%,28.6%,27.7%,51.6%,25.4%,14.7%
Raise Contribution Total (Pot%),32.9%,38.1%,57.5%,59.7%,91.3%,52.0%,39.0%


...

Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,54,407,829,633,406,38,82
Fold%,27.8%,33.9%,45.4%,50.4%,58.6%,68.4%,68.3%
Avg Size (Pot%),20.1%,31.8%,47.7%,69.6%,93.0%,108.7%,169.4%
Breakeven Fold%,16.6%,24.1%,32.3%,41.0%,48.2%,52.0%,61.7%
Fold Surplus,11.2%,9.8%,13.1%,9.4%,10.5%,16.4%,6.6%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,20,169,340,267,176,24,43
Continue%,60.0%,68.0%,51.8%,41.6%,34.7%,25.0%,25.6%
Call-Only%,45.0%,56.8%,45.9%,39.0%,32.4%,25.0%,23.3%
Raise%,15.0%,11.2%,5.9%,2.6%,2.3%,0.0%,2.3%
Avg Size (Pot%),18.9%,32.0%,47.7%,69.7%,94.1%,110.4%,165.9%
Avg Raise Size (Pot%),80.7%,98.8%,177.1%,184.3%,211.3%,0.0%,600.0%
Expected Call Contribution (Pot%),8.5%,18.2%,21.9%,27.1%,30.5%,27.6%,38.6%
Expected Raise Contribution (Pot%),12.1%,11.1%,10.4%,4.8%,4.8%,0.0%,14.0%
Raise Contribution Total (Pot%),20.6%,29.3%,32.3%,32.0%,35.3%,27.6%,52.5%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,20,169,340,267,176,24,43
Fold%,40.0%,32.0%,48.2%,58.4%,65.3%,75.0%,74.4%
Avg Size (Pot%),18.9%,32.0%,47.7%,69.7%,94.1%,110.4%,165.9%
Breakeven Fold%,15.7%,24.2%,32.2%,41.0%,48.4%,52.4%,61.5%
Fold Surplus,24.3%,7.8%,16.0%,17.4%,16.9%,22.6%,12.9%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,34,238,489,366,230,14,39
Continue%,79.4%,64.7%,56.6%,55.5%,46.5%,42.9%,38.5%
Call-Only%,64.7%,51.7%,47.9%,48.1%,39.1%,35.7%,20.5%
Raise%,14.7%,13.0%,8.8%,7.4%,7.4%,7.1%,17.9%
Avg Size (Pot%),20.8%,31.6%,47.7%,69.6%,92.3%,105.6%,173.2%
Avg Raise Size (Pot%),79.6%,103.9%,158.8%,256.3%,447.4%,200.0%,956.9%
Expected Call Contribution (Pot%),13.4%,16.3%,22.8%,33.4%,36.1%,37.7%,35.5%
Expected Raise Contribution (Pot%),11.7%,13.5%,14.0%,18.9%,33.1%,14.3%,171.8%
Raise Contribution Total (Pot%),25.2%,29.9%,36.8%,52.4%,69.2%,52.0%,207.3%


Bucket,"[0.00, 0.25)","[0.25, 0.40)","[0.40, 0.60)","[0.60, 0.80)","[0.80, 1.00)","[1.00, 1.25)",>=1.25
Events,34,238,489,366,230,14,39
Fold%,20.6%,35.3%,43.4%,44.5%,53.5%,57.1%,61.5%
Avg Size (Pot%),20.8%,31.6%,47.7%,69.6%,92.3%,105.6%,173.2%
Breakeven Fold%,17.1%,24.0%,32.3%,41.0%,47.9%,51.3%,61.9%
Fold Surplus,3.5%,11.3%,11.1%,3.5%,5.5%,5.9%,-0.4%


### Turn Nuts and Near-Nuts: First-Bet Tendencies

Evaluate how often players lead the turn when holding trips or better.


In [12]:
TURN_FIRST_ACTIONS_CACHE = PROJECT_ROOT / 'analysis' / 'cache' / 'turn_first_actions.json'
turn_first_action_records = load_turn_first_actions(
    DB_PATH, cache_path=TURN_FIRST_ACTIONS_CACHE, force=FORCE_RELOAD
)
turn_first_actions_df = pd.DataFrame(turn_first_action_records)

if turn_first_actions_df.empty:
    print('No turn first-action opportunities available.')
else:
    strong_categories = ['Trips/Set', 'Straight', 'Flush', 'Full House', 'Quads']
    strong_df = turn_first_actions_df[turn_first_actions_df['primary'].isin(strong_categories)].copy()
    if strong_df.empty:
        print('No turn opportunities with trips or better available.')
    else:
        position_order = ['Out of Position', 'In Position']
        strong_df['position'] = np.where(strong_df['in_position'], 'In Position', 'Out of Position')
        summary = (
            strong_df.groupby('position', sort=False)
            .agg(Opportunities=('bet_flag', 'count'), Bets=('bet_flag', 'sum'))
            .reindex(position_order)
        )
        summary['Bets'] = summary['Bets'].fillna(0.0)
        summary['Opportunities'] = summary['Opportunities'].fillna(0.0)
        summary['Bet %'] = np.where(
            summary['Opportunities'] > 0,
            summary['Bets'] / summary['Opportunities'] * 100,
            0.0,
        )
        summary = summary[['Opportunities', 'Bets', 'Bet %']]
        summary_styled = (
            summary.style
            .format({'Opportunities': '{:.0f}', 'Bets': '{:.0f}', 'Bet %': '{:.1f}%'})
            .set_caption('Turn First-Bet Frequency with Trips or Better')
        )
        display(summary_styled)

        bet_df = strong_df[strong_df['bet_flag']].copy()
        if bet_df.empty:
            print('No qualifying bets to bucket.')
        else:
            bucket_edges = [bucket[0] for bucket in TURN_BUCKETS] + [TURN_BUCKETS[-1][1]]
            bucket_labels = [bucket[2] for bucket in TURN_BUCKETS]
            bet_df['bucket'] = pd.cut(
                pd.to_numeric(bet_df['ratio'], errors='coerce'),
                bins=bucket_edges,
                labels=bucket_labels,
                right=False,
                include_lowest=True,
            )
            bucket_counts = (
                bet_df.groupby(['bucket', 'position'], observed=False)
                .size()
                .unstack('position', fill_value=0.0)
                .reindex(bucket_labels, fill_value=0.0)
                .reindex(columns=position_order, fill_value=0.0)
            )
            column_totals = bucket_counts.sum(axis=0).replace(0.0, np.nan)
            bucket_pct = bucket_counts.div(column_totals, axis=1) * 100.0
            bucket_pct = bucket_pct.fillna(0.0)
            bucket_styled = (
                bucket_pct.style
                .format('{:.1f}%')
                .set_caption('Bet Size Distribution (Trips or Better)')
            )
            display(bucket_styled)

,Opportunities,Bets,Bet %
position,,,
Out of Position,1055,564,53.5%
In Position,602,462,76.7%


/tmp/ipykernel_6429/1263857639.py:51: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bet_df.groupby(['bucket', 'position'])


position,Out of Position,In Position
bucket,,
"[0.00, 0.25)",6.6%,4.8%
"[0.25, 0.40)",15.4%,12.8%
"[0.40, 0.60)",25.7%,27.3%
"[0.60, 0.80)",24.6%,26.8%
"[0.80, 1.00)",19.7%,17.7%
"[1.00, 1.25)",2.0%,3.0%
>=1.25,6.0%,7.6%
